# O que é Autoencoder?
Um autoencoder é uma rede neural que aprende a codificar os dados em uma representação comprimida (espaço latente) e depois tenta reconstruí-los o mais próximo possível dos dados originais.

Fontes: 
1. [Introdução aos codificadores automáticos](https://www.tensorflow.org/tutorials/generative/autoencoder?hl=pt-br)
2. [Cap 14 - Autoencoders](https://www.deeplearningbook.org/)
3. [IBM -What is an autoencoder?](https://www.ibm.com/topics/autoencoder)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

# Carregamento do dataset 
O dataset **Fashion MNIST** é carregado, ele possui imagens de roupas e acessórios, com 60.0000 imagens para treino e 10.000 para testes.
* As variáveis x_train e x_test contêm essas respectivas imagens
* O uso de _ ignora os rótulos (labels)

In [ ]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()

As imagens são originalmente representadas como arrays de inteiros de 8 bits (valores entre 0 e 255), e são convertidas para float32 (ponto flutuante de 32 bits).
* Elas são normalizadas dividindo cada valor de pixel por 255, para que os valores fiquem no intervalo de [0,1].
* A normalização facilita o treinamento de modelos de aprendizado de máquina

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Implementa o autoencoder
O código a seguir implementa o autoencoder utilizando a API **Model** do TensorFlow/Keras. 
* latent_dim: define o tamanho de espaço latente = número de unidades (neurônios) na camada de codificação comprimida. No caso 64.
* shape: a forma original dos dados de entrada [imagens]. No caso, (28,28)

In [ ]:
class Autoencoder(Model):
  def __init__(self, latent_dim, shape):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.shape = shape
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(tf.math.reduce_prod(shape).numpy(), activation='sigmoid'),
      layers.Reshape(shape)
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


# Encoder
O encoder é responsável por comprimir a entrada para o espaço latente.
* Flatten(): esta camada transforma as imagens 2D (de forma (28,28)) em vetores 1D de 784 (28*28) elementos.
* Dense(latent_dim, activation='relu'): após o flattening, os vetores são passados por uma camada densa com latent_dim unidades e ativação ReLU. -- Isso comprime as informações para um vetor de tamanho latent_dim (neste caso, 64 dimensões) 

![dense layer 2](https://pysource.com/wp-content/uploads/2022/08/flatten-and-dense-layers-computer-vision-with-keras-p-6-dense-layer-scheme-1024x723.jpg)
Fonte: https://pysource.com/2022/10/07/flatten-and-dense-layers-computer-vision-with-keras-p-6/

![dense layer](https://epynn.net/_images/Dense-01.svg)
Fonte: https://epynn.net/Dense.html

# Decoder
O decoder é responsável por reconstruir a imagem a partir da representação comprimida (espaço latente).
* Dense(tf.math.reduce_prod(shape).numpy(), activation='sigmoid'): A primeira camada densa reconstrói o veotr 1D, que tem o mesmo número de elementos que a imagem original (784), utilizando a função de ativação sigmoid -- isso se justifica pelo fato que os valores dos pixels foram normalizados entre 0 e 1.
* Reshape(shape): transforma o vetor 1D de volta para a forma original da imagem (28, 28)

# Método call
Método call define a passagem dos dados pelos autoencoder
* encoded:  a entrada é primeiro passada pelo encoder, onde é comprimida para o espaço latente.
* decoded: a representação comprimida é passada pelo decoder, que tenta reconstruir a imagem original.
* o método retorna a imagem reconstruída.

In [ ]:
shape = x_test.shape[1:]
latent_dim = 64
autoencoder = Autoencoder(latent_dim, shape)

# Inicialização do Autoencoder
* latent_dim = 64: O número de neurônios no espaço latente.
* shape = x_test.shape[1:]: A forma original das imagens de entrada (no caso do Fashion MNIST, (28, 28)).

# Resumo:
Este autoencoder comprime uma imagem de 28x28 pixels em um vetor de 64 dimensões (espaço latente) através do encoder e depois tenta reconstruir a imagem original a partir dessa representação comprimida através do decoder. A rede é projetada para treinar de forma que a saída seja o mais semelhante possível à entrada, forçando o autoencoder a aprender uma representação eficiente dos dados.

Compile(): é usado para configurar o modelo antes do treinamento
* otimizador: como os pesos do modelo serão atualizados durante o treinamento
* função de perda: a métrica que será usada para avaliar o quão bom o modelo está se saindo, ou seja, quão bem as saídas (imagens reconstruídas) se aproximam dos dados reais (imagens de entrada).

In [ ]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

optmizer = 'adam'
Adam (Adaptive Moment Estimation): combina as vantagens de dois outros algoritmos AdaGrad e RMSProp

loss=losses.MeanSquaredError()
função de perda
![mse](https://suboptimal.wiki/images/mse_5.jpg)
fonte: https://suboptimal.wiki/explanation/mse/
* n é o número de exemplos no conjunto de dados.
* yi​ é o valor real (neste caso, o pixel da imagem original).
* y^​i​ é o valor previsto pelo modelo (o pixel da imagem reconstruída).

A seguir é realizado o treinamento do modelo autoencoder, ajustando os pesos com base nos dados de treinamento. 
* autoencoder.fit(): método fit() em Keras é usado para treinar o modelo em um conjunto de dados de entrada.
*  shuffle=True: indica que o conjunto de dados será embaralhado antes de cada época. Isso evita que o modelo aprenda padrões indesejados ou artefatos que possam existir na sequência original dos dados

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test, x_test))

validation_data=(x_test, x_test)
* Durante o treinamento, além de ajustar os pesos com base nos dados de treinamento, é útil avaliar o desempenho do modelo em dados que não foram usados no treinamento para ver como ele se generaliza.

# Utilização do modelo autoencoder
autoencoder.encoder(x_test): Aqui, estamos utilizando a parte encoder do autoencoder, que é responsável por transformar (ou codificar) as imagens de entrada em uma representação compacta ou espaço latente.
* O x_test (imagens de teste) é passado para o encoder, que aplica a função Dense(latent_dim, activation='relu'), resultando em uma versão codificada ou comprimida das imagens.
* Essa codificação contém menos dimensões do que a imagem original e retém as informações mais importantes, ou seja, é a versão "compactada" das imagens originais.

* numpy(): Depois de passar as imagens pelo encoder, o resultado está em formato de tensor do TensorFlow. O método .numpy() converte esse tensor para um array do NumPy, que é um formato comum para manipulação de dados em Python.
* encoded_imgs: Este array contém as representações codificadas (ou "comprimidas") das imagens de teste. São representações no espaço latente, que têm um número reduzido de dimensões em comparação com as imagens originais.

In [ ]:
encoded_imgs = autoencoder.encoder(x_test).numpy()

autoencoder.decoder(encoded_imgs): Aqui, estamos passando as imagens codificadas (representações latentes) para a parte decoder do autoencoder, que é responsável por reconstruir as imagens originais a partir dessas representações compactadas.

* O decoder aplica uma camada densa seguida de uma camada de Reshape para reconstruir as imagens a partir do espaço latente.
* A saída do decoder deve ter o mesmo formato que as imagens originais (no caso do Fashion MNIST, uma imagem de 28x28 pixels).

* numpy(): Assim como no encoder, o método .numpy() converte o tensor de saída do TensorFlow para um array do NumPy.

* decoded_imgs: Este array contém as imagens reconstruídas a partir das representações latentes. Essas imagens são o resultado da tentativa do autoencoder de reconstruir as imagens de entrada a partir das informações comprimidas.

In [ ]:
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

# Comparação entre original e reconstruído
Esse código cria uma visualização comparativa entre as imagens originais e as imagens reconstruídas geradas pelo autoencoder para 10 exemplos do conjunto de teste x_test

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test[i])
  plt.title("original")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs[i])
  plt.title("reconstructed")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

# Aplicando ruído às imagens


In [ ]:
(x_train, _), (x_test, _) = fashion_mnist.load_data()

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


Expansão da Dimensão: 
* **tf.newaxis**: Adiciona uma nova dimensão ao array. Este comando é usado para expandir a forma das imagens de (num_images, height, width) para (num_images, height, width, 1). Essa nova dimensão é frequentemente necessária para que o modelo de autoencoder trate as imagens como dados de entrada com formato (altura, largura, canais), onde o número de canais é 1 (para imagens em escala de cinza).

In [ ]:
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

* noise_factor = 0.2: Define a intensidade do ruído a ser adicionado. Neste caso, um fator de 0.2 significa que o ruído adicionado terá um impacto considerável, mas não extremo, na imagem original

* tf.random.normal(shape=x_train.shape): Gera um tensor de números aleatórios com distribuição normal (ou gaussiana) com a mesma forma que x_train (ou x_test). O ruído gerado terá uma média de 0 e um desvio padrão de 1.
* x_train + noise_factor * tf.random.normal(...): Adiciona o ruído normalizado às imagens originais. O ruído é multiplicado pelo noise_factor para controlar a quantidade de ruído a ser adicionado. Isso cria a versão "ruidosa" das imagens.

In [ ]:
noise_factor = 0.2
x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape)
x_test_noisy = x_test + noise_factor * tf.random.normal(shape=x_test.shape)

tf.clip_by_value(...): Essa função é usada para garantir que todos os valores no tensor fiquem dentro de um intervalo específico. Aqui, ela assegura que todos os valores em x_train_noisy e x_test_noisy permaneçam entre 0 e 1. 
* Isso é importante porque, após adicionar ruído, alguns valores podem exceder 1 (o que não é válido para imagens normalizadas) ou ser menores que 0. Clipping impede que isso ocorra, mantendo os dados dentro da faixa esperada.

In [ ]:
x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)

In [ ]:
n = 10
plt.figure(figsize=(20, 2))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.title("original + noise")
    plt.imshow(tf.squeeze(x_test_noisy[i]))
    plt.gray()
plt.show()

In [ ]:
class Denoise(Model):
  def __init__(self):
    super(Denoise, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(28, 28, 1)),
      layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Denoise()